In [ ]:
from pyspark.sql import SparkSession
# Initialze Spark Session
spark = SparkSession.builder.getOrCreate()

In [ ]:
import pyspark.sql.functions as F
from re import sub

In [ ]:
# Read in metadata table.
index = spark \
        .read \
        .option("header", True) \
        .csv("s3://w251-asl-data/raw-data/train.csv")

index.show(3, truncate=False)

In [ ]:
# Read in raw data, manipulate to include filename as join key, join to index table, drop any rows missing data. 
data = spark.read.parquet("s3://w251-asl-data/raw-data/train_landmark_files/*/*") \
        .withColumn("path", F.input_file_name()) \
        .withColumn("path", F.substring("path", 29, 1000)) \
        .join(index, ["path"], "left") \
        .na.drop() \
        .drop(F.col("z")) \
        .cache()
        
data.show(3, truncate=False)

In [ ]:
# Get max frame length
data.select(F.max(F.col("frame"))).show()

In [ ]:
data.select("path").distinct().show()

In [ ]:
sample = spark.read.parquet("s3://w251-asl-data/raw-data/train_landmark_files/4718/2725590426.parquet", 
                           "s3://w251-asl-data/raw-data/train_landmark_files/16069/100015657.parquet",) \
        .withColumn("path", F.input_file_name()) \
        .withColumn("path", F.substring("path", 29, 1000)) \
        .join(index, ["path"], "left") \
        .na.drop() \
        .drop(F.col("z")) \
        .cache()

# Concat type and landmark index
test = sample.withColumn("landmark", F.concat(F.col("type"), F.lit("-"), F.col("landmark_index"))) \
            .drop("path", "row_id", "type", "landmark_index") \
            .groupBy(["participant_id", "sequence_id", "frame"]) \
            .pivot("landmark").sum("x", "y") \ 
            

In [ ]:
for name in test.columns:
    test = test.withColumnRenamed(name, name.replace("_sum", "-"))

In [ ]:
test.show()

In [ ]:
len(test.columns)